In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
import random
from joblib import Parallel, delayed
from multiprocessing import cpu_count
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import pyreadr

#### In this notebook, we explore the idea that factor components are important for panel data models, and as a result TWFE based models have significant omitted variable bias. A direct way to demonstrate this idea is to extract the leading factor from the matrix of residuals of a TWFE regression, then show that i) it has predictive power on outcomes of a held-out set/is correlated on the regional level and ii) it is also correlated with the treatment assignment. We start with the CPS data and then perform similar procedures on DiD datasets used in some previous works.

## CPS Data

In [2]:
df = pd.read_csv('CPS.csv',sep=';')
Y_true_full = np.reshape(df['log_wage'].values, (40,-1))
Y_true_full = Y_true_full.T
Y_true_full /= np.std(Y_true_full)
Y_true_full -= np.mean(Y_true_full)
N_total,T_total = Y_true_full.shape

In [3]:
df

,state,year,log_wage,hours,urate,min_wage,open_carry,abort_ban
0,AK,1979,-0.759183,37.504,0.073,False,False,False
1,AL,1979,-2.269627,36.289,0.065,False,False,False
2,AR,1979,-2.225654,37.182,0.060,False,False,False
3,AZ,1979,-1.724803,38.692,0.045,False,False,False
4,CA,1979,-1.651739,36.578,0.060,False,False,False
...,...,...,...,...,...,...,...,...
1995,VT,2018,1.676063,38.760,0.027,True,True,True
1996,WA,2018,1.702448,36.485,0.033,True,False,True
1997,WI,2018,1.311645,35.916,0.009,False,True,False
1998,WV,2018,1.264739,36.563,0.055,False,True,False


In [4]:
Y_true_full.shape

(50, 40)

In [5]:
states = list(df['state'][:50])

## Treatment Vector

In [6]:
treatment = np.reshape(df['min_wage'].values, (40,-1))
treatment = treatment.T

In [7]:
np.argwhere(treatment==True)

array([[ 4, 39],
       [ 6, 39],
       [ 7, 39],
       [18, 39],
       [36, 39],
       [38, 39],
       [45, 39],
       [46, 39]])

In [8]:
Ds = np.argwhere(treatment==True)[:,0]
Ds

array([ 4,  6,  7, 18, 36, 38, 45, 46])

In [9]:
assignment_vector = np.zeros((N_total,))
assignment_vector[Ds] = 1

In [10]:
assignment_vector

array([0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.])

## MSE on Test Samples

### TWFE

In [11]:
def TWFE(Y, M):
    units, times = Y.shape
    unit_effect = cp.Variable((1,units))
    time_effect = cp.Variable((1,times))
    unit_effects = cp.kron(np.ones((times,1)),unit_effect).T
    time_effects = cp.kron(np.ones((units,1)),time_effect)
    mu = cp.Variable()

    objective = cp.sum_squares(cp.multiply(Y-unit_effects-time_effects-mu,M))/np.sum(M)
    #objective = cp.sum_squares(cp.multiply(Y-unit_effects-time_effects,M))/np.sum(M)

    constraints = []

    prob = cp.Problem(cp.Minimize(objective),
                      constraints)
    prob.solve()
    
    return objective.value, unit_effect.value.T+time_effect.value+mu.value

In [13]:
np.random.seed(0)
mask = np.random.binomial(1, 0.5, size=(Y_true_full.shape))

In [14]:
_, predicted_values = TWFE(Y_true_full, mask)

In [15]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))
#RMSE

0.13001274431507118

In [16]:
residuals = Y_true_full-predicted_values
U, S, VT = np.linalg.svd(residuals)
factor = np.outer(U[:, 0]*S[0],VT[0, :])

### TWFE Plus Unit Loading

In [17]:
def TWFE_plus_loading(Y, M, loading_factor):
    units, times = Y.shape
    unit_effect = cp.Variable((1,units))
    time_effect = cp.Variable((1,times))
    unit_effects = cp.kron(np.ones((times,1)),unit_effect).T
    time_effects = cp.kron(np.ones((units,1)),time_effect)
    loading_factor_coefficient = cp.Variable()
    mu = cp.Variable()

    #objective = cp.sum_squares(cp.multiply(Y-unit_effects-time_effects-mu-loading_factor_coefficient*loading_factor,M))/np.sum(M)
    objective = cp.sum_squares(cp.multiply(Y-unit_effects-time_effects-loading_factor_coefficient*loading_factor,M))/np.sum(M)
    constraints = []

    prob = cp.Problem(cp.Minimize(objective),
                      constraints)
    prob.solve()
    
    return objective.value, unit_effect.value.T+time_effect.value+loading_factor_coefficient.value*loading_factor

In [18]:
#_, _, predicted_values = TWFE_one_factor(Y_true_full, mask)
#_, predicted_values = TWFE_plus_loading(Y_true_full, mask, unit_loadings.dot(time_factors))
_, predicted_values = TWFE_plus_loading(Y_true_full, mask, factor)

In [19]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

0.10484644233495499

## Unit Fixed Effects across Time

In [20]:
residuals = Y_true_full-np.mean(Y_true_full,axis=0)
indicator = np.zeros((T_total,))
indicator[20:] = 1
effects = []
stds = []
p_values = []
for i in range(N_total):
    model = sm.OLS(residuals[i,:], indicator).fit()
    print(model.summary())
    effects.append(model.params[0])
    stds.append(model.bse[0])
    p_values.append(model.pvalues[0])

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.086
Model:                            OLS   Adj. R-squared (uncentered):              0.063
Method:                 Least Squares   F-statistic:                              3.680
Date:                Tue, 19 Aug 2025   Prob (F-statistic):                      0.0624
Time:                        12:12:44   Log-Likelihood:                         -35.645
No. Observations:                  40   AIC:                                      73.29
Df Residuals:                      39   BIC:                                      74.98
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [21]:
result = pd.DataFrame({'state': states, 'effct difference': effects, 'std': stds, 'p values': p_values})

In [22]:
result

,state,effct difference,std,p values
0,AK,0.256258,0.133585,6.241311e-02
1,AL,-0.150996,0.040907,6.809744e-04
2,AR,-0.236294,0.049481,2.536904e-05
3,AZ,0.074319,0.033125,3.061112e-02
4,CA,0.231960,0.070824,2.220110e-03
5,CO,0.229738,0.039112,7.752249e-07
6,CT,0.373815,0.066891,1.930645e-06
7,DE,0.239007,0.037372,1.463600e-07
8,FL,0.038430,0.016414,2.441892e-02
9,GA,0.039771,0.023139,9.357590e-02


In [23]:
result.round(4).to_csv('effect_difference.csv', index=False)

In [24]:
np.mean(result['p values']<0.05)

0.76

## CPS urate

In [25]:
df = pd.read_csv('CPS.csv',sep=';')
Y_true_full = np.reshape(df['urate'].values, (40,-1))
Y_true_full = Y_true_full.T
Y_true_full /= np.std(Y_true_full)
Y_true_full -= np.mean(Y_true_full)
N_total,T_total = Y_true_full.shape

In [26]:
df

,state,year,log_wage,hours,urate,min_wage,open_carry,abort_ban
0,AK,1979,5.5881,37.504,1.106681,False,False,False
1,AL,1979,4.9183,36.289,0.722542,False,False,False
2,AR,1979,4.9378,37.182,0.482455,False,False,False
3,AZ,1979,5.1599,38.692,-0.237806,False,False,False
4,CA,1979,5.1923,36.578,0.482455,False,False,False
...,...,...,...,...,...,...,...,...
1995,VT,2018,6.6680,38.760,-1.102119,True,True,True
1996,WA,2018,6.6797,36.485,-0.814015,True,False,True
1997,WI,2018,6.5064,35.916,-1.966433,False,True,False
1998,WV,2018,6.4856,36.563,0.242368,False,True,False


In [27]:
Y_true_full.shape

(50, 40)

In [28]:
states = list(df['state'][:50])

## MSE on Test Samples

### TWFE

In [29]:
np.random.seed(0)
mask = np.random.binomial(1, 0.5, size=(Y_true_full.shape))

In [30]:
_, predicted_values = TWFE(Y_true_full, mask)

In [31]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))
#RMSE

0.6952404370737674

In [32]:
residuals = Y_true_full-predicted_values
U, S, VT = np.linalg.svd(residuals)
factor = np.outer(U[:, 0]*S[0],VT[0, :])

### TWFE Plus Unit Loading

In [33]:
#_, _, predicted_values = TWFE_one_factor(Y_true_full, mask)
#_, predicted_values = TWFE_plus_loading(Y_true_full, mask, unit_loadings.dot(time_factors))
_, predicted_values = TWFE_plus_loading(Y_true_full, mask, factor)

In [35]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

0.6183776323919402

## Unit Fixed Effects across Time

In [36]:
residuals = Y_true_full-np.mean(Y_true_full,axis=0)
indicator = np.zeros((T_total,))
indicator[20:] = 1
effects = []
stds = []
p_values = []
for i in range(N_total):
    model = sm.OLS(residuals[i,:], indicator).fit()
    print(model.summary())
    effects.append(model.params[0])
    stds.append(model.bse[0])
    p_values.append(model.pvalues[0])

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.070
Model:                            OLS   Adj. R-squared (uncentered):              0.046
Method:                 Least Squares   F-statistic:                              2.915
Date:                Tue, 19 Aug 2025   Prob (F-statistic):                      0.0957
Time:                        12:41:34   Log-Likelihood:                         -50.246
No. Observations:                  40   AIC:                                      102.5
Df Residuals:                      39   BIC:                                      104.2
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [37]:
result = pd.DataFrame({'state': states, 'effct difference': effects, 'std': stds, 'p values': p_values})

In [38]:
result

,state,effct difference,std,p values
0,AK,0.328535,0.192437,0.095729
1,AL,0.595032,0.224553,0.011571
2,AR,0.208492,0.189245,0.277343
3,AZ,0.062038,0.114654,0.591522
4,CA,0.779899,0.137960,0.000002
5,CO,-0.190053,0.140165,0.182924
6,CT,0.057237,0.178580,0.750293
7,DE,-0.084415,0.151397,0.580324
8,FL,0.138866,0.104262,0.190627
9,GA,0.479790,0.152686,0.003196


In [39]:
result.round(4).to_csv('effect_difference.csv', index=False)

In [40]:
np.mean(result['p values']<0.05)

0.58

## PENN Data

In [37]:
df = pd.read_csv('PENN.csv',sep=';')
Y_true_full = np.reshape(df['log_gdp'].values, (48,-1))
Y_true_full = Y_true_full.T
Y_true_full /= np.std(Y_true_full)
Y_true_full -= np.mean(Y_true_full)
N_total,T_total = Y_true_full.shape

democracy = np.reshape(df['dem'].values, (48,-1))
democracy = democracy.T
df

,country,year,log_gdp,dem,educ
0,Argentina,1960,0.459565,False,False
1,Australia,1960,0.945638,False,False
2,Austria,1960,0.637185,False,False
3,Burundi,1960,-1.749135,False,False
4,Belgium,1960,0.627998,False,False
...,...,...,...,...,...
5323,United States,2007,1.895453,True,True
5324,Venezuela (Bolivarian Republic of),2007,1.019204,False,False
5325,South Africa,2007,0.637374,False,False
5326,Zambia,2007,-0.610057,False,False


In [38]:
Y_true_full

array([[ 0.45956458,  0.47298454,  0.46506122, ...,  0.77319728,
         0.82957345,  0.89354427],
       [ 0.94563819,  0.93794953,  0.9738106 , ...,  1.75882824,
         1.77625286,  1.79138443],
       [ 0.63718524,  0.67555933,  0.68700114, ...,  1.67223843,
         1.69768914,  1.72578502],
       ...,
       [ 0.24590107,  0.25456422,  0.28388819, ...,  0.56562785,
         0.6022413 ,  0.6373743 ],
       [-0.58751627, -0.600719  , -0.61821316, ..., -0.6959263 ,
        -0.65467008, -0.61005742],
       [-1.31389098, -1.29061389, -1.30024676, ..., -1.42821256,
        -1.47123725, -1.51233664]])

In [39]:
np.random.seed(0)
mask = np.random.binomial(1, 0.5, size=(Y_true_full.shape))

## MSE on Test Samples

### TWFE

In [40]:
#_, predicted_values = TWFE_treatment(Y_true_full, mask, W_true_full, np.zeros(Y_true_full.shape))
_, predicted_values = TWFE(Y_true_full, mask)

In [41]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

0.2462177104882635

In [42]:
residuals = Y_true_full-predicted_values
U, S, VT = np.linalg.svd(residuals)
factor = np.outer(U[:, 0]*S[0],VT[0, :])

### TWFE Plus Unit Loading

In [43]:
#_, _, predicted_values = TWFE_one_factor(Y_true_full, mask)
#_, predicted_values = TWFE_treatment(Y_true_full, mask, W_true_full, factor)
_, predicted_values = TWFE_plus_loading(Y_true_full, mask, factor)

In [44]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

0.10512561789611187

## Unit Fixed Effects across Time

In [45]:
residuals = Y_true_full-np.mean(Y_true_full,axis=0)
indicator = np.zeros((T_total,))
indicator[24:] = 1
effects = []
stds = []
p_values = []
for i in range(N_total):
    model = sm.OLS(residuals[i,:], indicator).fit()
    print(model.summary())
    effects.append(model.params[0])
    stds.append(model.bse[0])
    p_values.append(model.pvalues[0])

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.303
Model:                            OLS   Adj. R-squared (uncentered):              0.288
Method:                 Least Squares   F-statistic:                              20.43
Date:                Sun, 29 Jun 2025   Prob (F-statistic):                    4.18e-05
Time:                        06:40:02   Log-Likelihood:                         -39.295
No. Observations:                  48   AIC:                                      80.59
Df Residuals:                      47   BIC:                                      82.46
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [46]:
result = pd.DataFrame({'country': df['country'].unique(), 'effct difference': effects, 'std': stds, 'p values': p_values})

In [47]:
pd.set_option('display.max_rows', None)
result

,country,effct difference,std,p values
0,Argentina,0.511510,0.113179,4.181439e-05
1,Australia,1.416085,0.197569,4.555074e-09
2,Austria,1.352168,0.170414,3.189712e-10
3,Burundi,-1.687552,0.213929,3.740078e-10
4,Belgium,1.305372,0.170017,7.736187e-10
5,Benin,-1.183072,0.161020,2.435234e-09
6,Burkina Faso,-1.663004,0.227436,2.754527e-09
7,Bangladesh,-1.200502,0.156308,7.669590e-10
8,Bolivia (Plurinational State of),-0.434403,0.022098,2.595596e-24
9,Brazil,0.454674,0.058953,6.862429e-10


In [48]:
result.round(4).to_csv('effect_difference_PENN.csv', index=False)

In [49]:
np.mean(result['p values']<0.05)

0.9099099099099099

## Germany Data

In [87]:
df = pd.read_stata('germany.dta')
df

,index,country,year,gdp,infrate,trade,schooling,invest60,invest70,invest80,industry
0,1.0,USA,1960.0,2879,NaN,9.693181,43.799999,NaN,NaN,NaN,NaN
1,1.0,USA,1961.0,2929,1.075182,9.444654,NaN,NaN,NaN,NaN,NaN
2,1.0,USA,1962.0,3103,1.116071,9.429324,NaN,NaN,NaN,NaN,NaN
3,1.0,USA,1963.0,3227,1.214128,9.470706,NaN,NaN,NaN,NaN,NaN
4,1.0,USA,1964.0,3420,1.308615,9.725879,NaN,NaN,NaN,NaN,NaN
5,1.0,USA,1965.0,3667,1.668461,9.730347,43.799999,NaN,NaN,NaN,NaN
6,1.0,USA,1966.0,3974,2.991000,10.097592,NaN,NaN,NaN,NaN,NaN
7,1.0,USA,1967.0,4154,2.775636,10.208102,NaN,NaN,NaN,NaN,NaN
8,1.0,USA,1968.0,4494,4.217721,10.593937,NaN,NaN,NaN,NaN,NaN
9,1.0,USA,1969.0,4805,5.414701,10.635633,NaN,NaN,NaN,NaN,NaN


In [88]:
df = pd.read_stata('germany.dta')
Y_true_full = np.reshape(df['gdp'].copy().values, (17,-1))
#Y_true_full = np.log(np.delete(Y_true_full,6,0))
Y_true_full = np.log(Y_true_full)
Y_true_full /= np.std(Y_true_full)
Y_true_full -= np.mean(Y_true_full)
N_total,T_total = Y_true_full.shape

In [89]:
N_total, T_total

(17, 44)

In [90]:
Y_true_full

array([[-1.18906747, -1.17027252, -1.10727906, -1.06450695, -1.00109942,
        -0.92497964, -0.83721711, -0.7888615 , -0.70298502, -0.62994292,
        -0.58673711, -0.51021801, -0.41737746, -0.30618868, -0.22689907,
        -0.14115776, -0.03328487,  0.07266547,  0.19482483,  0.30409972,
         0.38591851,  0.50036386,  0.53270886,  0.61329066,  0.71998456,
         0.78719689,  0.83803095,  0.89388276,  0.96486969,  1.03310632,
         1.08233284,  1.10310054,  1.14866559,  1.18803143,  1.24090327,
         1.27729008,  1.32530676,  1.37882867,  1.42325366,  1.47430174,
         1.52515284,  1.54818895,  1.57380047,  1.61431311],
       [-1.50372868, -1.4747778 , -1.44561532, -1.39409386, -1.32840152,
        -1.27543019, -1.22178088, -1.1725109 , -1.08292786, -1.01039501,
        -0.94056968, -0.87068679, -0.79017619, -0.6580164 , -0.57868752,
        -0.48625717, -0.39608391, -0.30197504, -0.19212493, -0.07716073,
        -0.00703674,  0.07445427,  0.16101153,  0.24079745,  0.

In [91]:
np.random.seed(0)
mask = np.random.binomial(1, 0.5, size=(Y_true_full.shape))

## MSE on Test Samples

### TWFE

In [92]:
#_, predicted_values = TWFE_treatment(Y_true_full, mask, W_true_full, np.zeros(Y_true_full.shape))
_, predicted_values = TWFE(Y_true_full, mask)

In [93]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

0.1362199881846545

In [94]:
residuals = Y_true_full-predicted_values
U, S, VT = np.linalg.svd(residuals)
factor = np.outer(U[:, 0]*S[0],VT[0, :])

### TWFE Plus Unit Loading

In [95]:
#_, _, predicted_values = TWFE_one_factor(Y_true_full, mask)
#_, predicted_values = TWFE_treatment(Y_true_full, mask, W_true_full, factor)
_, predicted_values = TWFE_plus_loading(Y_true_full, mask, factor)

In [96]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

0.0613810004083193

## Unit Fixed Effects across Time

In [97]:
residuals = Y_true_full-np.mean(Y_true_full,axis=0)
indicator = np.zeros((T_total,))
indicator[22:] = 1
effects = []
stds = []
p_values = []
for i in range(N_total):
    model = sm.OLS(residuals[i,:], indicator).fit()
    print(model.summary())
    effects.append(model.params[0])
    stds.append(model.bse[0])
    p_values.append(model.pvalues[0])

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.405
Model:                            OLS   Adj. R-squared (uncentered):              0.391
Method:                 Least Squares   F-statistic:                              29.31
Date:                Sun, 29 Jun 2025   Prob (F-statistic):                    2.59e-06
Time:                        08:33:03   Log-Likelihood:                         -4.7981
No. Observations:                  44   AIC:                                      11.60
Df Residuals:                      43   BIC:                                      13.38
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [98]:
result = pd.DataFrame({'country': df['country'].unique(), 'effct difference': effects, 'std': stds, 'p values': p_values})

In [99]:
pd.set_option('display.max_rows', None)
result

,country,effct difference,std,p values
0,USA,0.315065,0.058197,2.585669e-06
1,UK,-0.040766,0.016860,1.991948e-02
2,Austria,0.105330,0.008576,1.185571e-15
3,Belgium,0.063042,0.007046,2.256344e-11
4,Denmark,0.103203,0.034723,4.828022e-03
5,France,0.054116,0.007785,1.515488e-08
6,West Germany,0.127287,0.031038,1.794263e-04
7,Italy,0.023820,0.009469,1.570365e-02
8,Netherlands,0.093671,0.021604,8.602362e-05
9,Norway,0.155011,0.021078,3.967249e-09


In [100]:
result.round(4).to_csv('effect_difference_Germany.csv', index=False)

In [101]:
np.mean(result['p values']<0.05)

0.8823529411764706

## Basque Data

In [117]:
result = pyreadr.read_r('basque.rda')
df = result['basque']
df

,regionno,regionname,year,gdpcap,sec.agriculture,sec.energy,sec.industry,sec.construction,sec.services.venta,sec.services.nonventa,school.illit,school.prim,school.med,school.high,school.post.high,popdens,invest
rownames,,,,,,,,,,,,,,,,,
1,1.0,Spain (Espana),1955.0,2.354542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Spain (Espana),1956.0,2.480149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Spain (Espana),1957.0,2.603613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,Spain (Espana),1958.0,2.637104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,Spain (Espana),1959.0,2.669880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,Spain (Espana),1960.0,2.869966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,Spain (Espana),1961.0,3.047486,19.540001,4.710000,26.420000,6.27,36.619999,6.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,Spain (Espana),1962.0,3.273279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.0,Spain (Espana),1963.0,3.493502,19.049999,4.310000,26.049999,6.83,38.000000,5.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
Y_true_full = df['gdpcap'].copy().values
Y_true_full = np.reshape(Y_true_full, (18,-1))
Y_true_full /= np.std(Y_true_full)
Y_true_full -= np.mean(Y_true_full)
N_total,T_total = Y_true_full.shape

In [122]:
N_total,T_total

(18, 43)

In [123]:
Y_true_full

array([[-1.35662208e+00, -1.30057728e+00, -1.24548839e+00,
        -1.23054522e+00, -1.21592059e+00, -1.12664401e+00,
        -1.04743575e+00, -9.46688817e-01, -8.48427197e-01,
        -8.00857630e-01, -7.55932545e-01, -7.01194085e-01,
        -6.49354958e-01, -5.44912248e-01, -4.39258581e-01,
        -3.58425341e-01, -2.79280867e-01, -1.43103786e-01,
        -9.57108923e-03,  4.98828477e-02,  1.06023233e-01,
         1.34953755e-01,  1.62641622e-01,  1.43524573e-01,
         1.30397548e-01,  1.40911886e-01,  1.62832785e-01,
         1.91986362e-01,  2.22446186e-01,  2.71831896e-01,
         3.21886578e-01,  4.91486797e-01,  6.61787687e-01,
         8.24537552e-01,  9.87669742e-01,  1.07082883e+00,
         1.15271375e+00,  1.08867133e+00,  1.02494783e+00,
         1.11320511e+00,  1.20464817e+00,  1.30692431e+00,
         1.46145379e+00],
       [-1.65370104e+00, -1.62257215e+00, -1.59172995e+00,
        -1.58051465e+00, -1.56923557e+00, -1.51029133e+00,
        -1.45717775e+00, -1.38

In [124]:
np.random.seed(0)
mask = np.random.binomial(1, 0.5, size=(Y_true_full.shape))

## MSE on Test Samples

### TWFE

In [ ]:
#_, predicted_values = TWFE_treatment(Y_true_full, mask, W_true_full, np.zeros(Y_true_full.shape))
_, predicted_values = TWFE(Y_true_full, mask)

In [ ]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

In [ ]:
residuals = Y_true_full-predicted_values
U, S, VT = np.linalg.svd(residuals)
factor = np.outer(U[:, 0]*S[0],VT[0, :])

### TWFE Plus Unit Loading

In [ ]:
#_, _, predicted_values = TWFE_one_factor(Y_true_full, mask)
#_, predicted_values = TWFE_treatment(Y_true_full, mask, W_true_full, factor)
_, predicted_values = TWFE_plus_loading(Y_true_full, mask, factor)

In [ ]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

## Unit Fixed Effects across Time

In [77]:
residuals = Y_true_full-np.mean(Y_true_full,axis=0)
indicator = np.zeros((T_total,))
indicator[22:] = 1
effects = []
stds = []
p_values = []
for i in range(N_total):
    model = sm.OLS(residuals[i,:], indicator).fit()
    print(model.summary())
    effects.append(model.params[0])
    stds.append(model.bse[0])
    p_values.append(model.pvalues[0])

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.195
Model:                            OLS   Adj. R-squared (uncentered):              0.176
Method:                 Least Squares   F-statistic:                              10.20
Date:                Sun, 29 Jun 2025   Prob (F-statistic):                     0.00266
Time:                        08:30:01   Log-Likelihood:                          105.06
No. Observations:                  43   AIC:                                     -208.1
Df Residuals:                      42   BIC:                                     -206.4
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [81]:
result = pd.DataFrame({'regionname': df['regionname'].unique(), 'effct difference': effects, 'std': stds, 'p values': p_values})

In [82]:
pd.set_option('display.max_rows', None)
result

,regionname,effct difference,std,p values
0,Spain (Espana),-0.014825,0.004641,2.660091e-03
1,Andalucia,-0.866042,0.084610,5.570597e-13
2,Aragon,0.202551,0.012420,8.874056e-20
3,Principado De Asturias,-0.228598,0.032615,1.411264e-08
4,Baleares (Islas),1.452851,0.130648,4.290525e-14
5,Canarias,-0.193594,0.041309,2.926355e-05
6,Cantabria,-0.070559,0.022023,2.589189e-03
7,Castilla Y Leon,-0.388759,0.061607,1.420574e-07
8,Castilla-La Mancha,-0.690364,0.097680,1.163654e-08
9,Cataluna,0.765082,0.101818,2.697143e-09


In [83]:
result.round(4).to_csv('effect_difference_basque.csv', index=False)

In [85]:
np.mean(result['p values']<0.05)

1.0

## Smoking

In [3]:
df = pd.read_csv('california_prop99.csv',sep=';')
df

,State,Year,PacksPerCapita,treated
0,Alabama,1970,89.800003,0
1,Arkansas,1970,100.300003,0
2,Colorado,1970,124.800003,0
3,Connecticut,1970,120.000000,0
4,Delaware,1970,155.000000,0
...,...,...,...,...
1204,Virginia,2000,96.699997,0
1205,West Virginia,2000,107.900002,0
1206,Wisconsin,2000,80.099998,0
1207,Wyoming,2000,90.500000,0


In [4]:
Y_true_full = df['PacksPerCapita'].copy().values
Y_true_full = np.reshape(Y_true_full, (31,-1))
#Y_true_full = Y_true_full.T[:-1,:]
Y_true_full = Y_true_full.T
Y_true_full /= np.std(Y_true_full)
Y_true_full -= np.mean(Y_true_full)
N_total,T_total = Y_true_full.shape

In [5]:
N_total,T_total

(39, 31)

In [6]:
Y_true_full

array([[-0.88823801, -0.71726579, -0.54324054, ..., -0.3875337 ,
        -0.55545289, -0.69284132],
       [-0.56766501, -0.45164825, -0.45775431, ..., -0.28678209,
        -0.43027658, -0.59514274],
       [ 0.18033867,  0.2017101 ,  0.47038091, ..., -1.15080275,
        -1.19965192, -1.40115491],
       ...,
       [-0.38142741, -0.41195817, -0.30815353, ..., -0.92182204,
        -1.05310417, -1.18438654],
       [ 0.40626612,  0.39100074,  0.64440616, ..., -0.48828507,
        -0.43027658, -0.86686657],
       [ 0.1253832 ,  0.06432167,  0.14064858, ..., -2.03314171,
        -2.18884855, -2.35982089]])

In [7]:
np.random.seed(0)
mask = np.random.binomial(1, 0.5, size=(Y_true_full.shape))

## MSE on Test Samples

### TWFE

In [12]:
#_, predicted_values = TWFE_treatment(Y_true_full, mask, W_true_full, np.zeros(Y_true_full.shape))
_, predicted_values = TWFE(Y_true_full, mask)

In [13]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

0.36907672290444177

In [14]:
residuals = Y_true_full-predicted_values
U, S, VT = np.linalg.svd(residuals)
factor = np.outer(U[:, 0]*S[0],VT[0, :])

### TWFE Plus Unit Loading

In [15]:
#_, _, predicted_values = TWFE_one_factor(Y_true_full, mask)
#_, predicted_values = TWFE_treatment(Y_true_full, mask, W_true_full, factor)
_, predicted_values = TWFE_plus_loading(Y_true_full, mask, factor)

In [16]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

0.20270478050275428

## Unit Fixed Effects across Time

In [20]:
residuals = Y_true_full-np.mean(Y_true_full,axis=0)
indicator = np.zeros((T_total,))
indicator[15:] = 1
effects = []
stds = []
p_values = []
for i in range(N_total):
    model = sm.OLS(residuals[i,:], indicator).fit()
    print(model.summary())
    effects.append(model.params[0])
    stds.append(model.bse[0])
    p_values.append(model.pvalues[0])

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.008
Model:                            OLS   Adj. R-squared (uncentered):             -0.026
Method:                 Least Squares   F-statistic:                             0.2290
Date:                Tue, 15 Jul 2025   Prob (F-statistic):                       0.636
Time:                        12:18:35   Log-Likelihood:                         -21.540
No. Observations:                  31   AIC:                                      45.08
Df Residuals:                      30   BIC:                                      46.51
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [24]:
result = pd.DataFrame({'State': df['State'].unique(), 'effct difference': effects, 'std': stds, 'p values': p_values})

In [25]:
pd.set_option('display.max_rows', None)
result

,State,effct difference,std,p values
0,Alabama,0.058953,0.123194,6.357388e-01
1,Arkansas,0.293276,0.094896,4.286793e-03
2,Colorado,-0.476099,0.036777,8.195435e-14
3,Connecticut,-0.538114,0.113035,4.576828e-05
4,Delaware,0.802186,0.121941,2.802164e-07
5,Georgia,0.157796,0.056810,9.350134e-03
6,Idaho,-0.651651,0.098990,2.767651e-07
7,Illinois,-0.362181,0.046707,1.188507e-08
8,Indiana,0.816306,0.110304,3.025487e-08
9,Iowa,-0.279557,0.086886,3.095626e-03


In [26]:
result.round(4).to_csv('effect_difference_smoking.csv', index=False)

In [27]:
np.mean(result['p values']<0.05)

0.7948717948717948

## Boatlift

In [31]:
df = pd.read_stata('aux_may-org.dta')
df = df[['smsarank','year','loguearnhre']]
df = df.dropna()

In [32]:
cities = list(df['smsarank'].unique())
cities.index('Miami')

25

In [33]:
Y_true_full = np.reshape(df.copy().groupby(['smsarank','year'],observed=False).mean().values,(-1,19))
#Y_true_full

In [34]:
#Y_true_full = np.delete(Y_true_full,25,0)
Y_true_full /= np.std(Y_true_full)
Y_true_full -= np.mean(Y_true_full)
N_total,T_total = Y_true_full.shape

In [35]:
Y_true_full.shape

(44, 19)

In [36]:
np.random.seed(0)
mask = np.random.binomial(1, 0.5, size=(Y_true_full.shape))

## MSE on Test Samples

### TWFE

In [37]:
#_, predicted_values = TWFE_treatment(Y_true_full, mask, W_true_full, np.zeros(Y_true_full.shape))
_, predicted_values = TWFE(Y_true_full, mask)

In [38]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

0.4795093100838155

In [39]:
residuals = Y_true_full-predicted_values
U, S, VT = np.linalg.svd(residuals)
factor = np.outer(U[:, 0]*S[0],VT[0, :])

### TWFE Plus Unit Loading

In [40]:
#_, _, predicted_values = TWFE_one_factor(Y_true_full, mask)
#_, predicted_values = TWFE_treatment(Y_true_full, mask, W_true_full, factor)
_, predicted_values = TWFE_plus_loading(Y_true_full, mask, factor)

In [41]:
np.sqrt(np.mean(np.square(Y_true_full[mask == 0]-predicted_values[mask == 0])))

0.3912346643482526

## Unit Fixed Effects across Time

In [43]:
residuals = Y_true_full-np.mean(Y_true_full,axis=0)
indicator = np.zeros((T_total,))
indicator[9:] = 1
effects = []
stds = []
p_values = []
for i in range(N_total):
    model = sm.OLS(residuals[i,:], indicator).fit()
    print(model.summary())
    effects.append(model.params[0])
    stds.append(model.bse[0])
    p_values.append(model.pvalues[0])

/home/users/zhaonanq/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/home/users/zhaonanq/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/home/users/zhaonanq/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/home/users/zhaonanq/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/home/users/zhaonanq/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.440
Model:                            OLS   Adj. R-squared (uncentered):              0.409
Method:                 Least Squares   F-statistic:                              14.16
Date:                Tue, 15 Jul 2025   Prob (F-statistic):                     0.00142
Time:                        12:50:02   Log-Likelihood:                         -18.987
No. Observations:                  19   AIC:                                      39.97
Df Residuals:                      18   BIC:                                      40.92
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

/home/users/zhaonanq/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/home/users/zhaonanq/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/home/users/zhaonanq/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/home/users/zhaonanq/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/home/users/zhaonanq/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py

In [45]:
result = pd.DataFrame({'city': df['smsarank'].unique(), 'effct difference': effects, 'std': stds, 'p values': p_values})

In [46]:
pd.set_option('display.max_rows', None)
result

,city,effct difference,std,p values
0,New York City,0.803661,0.213547,1.422858e-03
1,Los Angeles,0.390589,0.086716,2.744243e-04
2,Chicago,0.420092,0.208967,5.962471e-02
3,Philadelphia,0.178854,0.091848,6.727497e-02
4,Detroit,0.257971,0.282419,3.730870e-01
5,San Francisco-Oakland,1.444092,0.363992,9.031368e-04
6,Washington DC,1.286762,0.340935,1.388952e-03
7,Boston,0.718447,0.152092,1.694040e-04
8,"Nassau-Suffolk, NY",1.327883,0.289808,2.312553e-04
9,Pittsburgh,-0.819120,0.103293,2.776543e-07


In [47]:
result.round(4).to_csv('effect_difference_smoking.csv', index=False)

In [48]:
np.mean(result['p values']<0.05)

0.6590909090909091